# Install Required Libraries

Installs the necessary Python libraries, including Pillow for image processing, Geopy for geographical computations, and gmplot for visualizing coordinates on a map.

In [1]:
%pip install pillow geopy gmplot

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Import Libraries

Imports the required libraries for working with image files and EXIF data.


In [2]:
import os
from PIL import Image
from PIL.ExifTags import TAGS

# Extract EXIF Data

This function extracts the EXIF metadata from an image file, which may include details such as camera settings, date/time, and GPS information.

In [3]:
def extract_exif(image_file):
    """
    Extracts the EXIF data from an image file.

    Args:
        image_file (str): The path to the image file.

    Returns:
        dict: A dictionary of EXIF tags and their values.
    """
    try:
        image = Image.open(image_file)
        exif = {}
        if image._getexif() is not None:
            for tag, value in image._getexif().items():
                if tag in TAGS:
                    exif[TAGS[tag]] = value
        return exif
    except Exception as e:
        print(f"Error reading EXIF data: {e}")
        return {}

# Get GPS Information

This function extracts GPS data (`latitude` and `longitude`) from the EXIF metadata of an image. Returns `None` if no GPS data is found.

In [4]:
def get_gps_info(exif_data):
    """
    Extracts GPS information from EXIF data.

    Args:
        exif_data (dict): A dictionary of EXIF tags and their values.

    Returns:
        dict or None: A dictionary containing GPS coordinates and references, or None if not available.
    """
    if "GPSInfo" in exif_data:
        gps_info = exif_data["GPSInfo"]
        return {
            "lat": gps_info[2],
            "lat_ref": gps_info[1],
            "lon": gps_info[4],
            "lon_ref": gps_info[3]
        }
    return None


# Convert GPS Coordinates to Degrees

Converts GPS coordinates stored in EXIF metadata into degrees (`float` format).
This is used to make the data usable for mapping or further calculations.

In [5]:
def convert_to_degrees(value):
    """
    Converts GPS coordinates stored in EXIF to degrees in float format.

    Args:
        value (tuple): The GPS coordinate as a tuple (degrees, minutes, seconds).

    Returns:
        float: The coordinate in degrees.
    """
    d = float(value[0])
    m = float(value[1])
    s = float(value[2])
    return d + (m / 60.0) + (s / 3600.0)

# Format GPS Coordinates

Formats raw GPS data into a human-readable string and computes `latitude` and `longitude` in float. Also adjusts for hemispherical references (`N/S`, `E/W`).

In [6]:
def format_gps_coordinates(gps_data):
    """
    Converts raw GPS data into a formatted string and calculates latitude and longitude in float.

    Args:
        gps_data (dict): A dictionary containing raw GPS data.

    Returns:
        tuple: A formatted string and float values of latitude and longitude.
    """
    lat = convert_to_degrees(gps_data["lat"])
    lon = convert_to_degrees(gps_data["lon"])

    # Adjust the sign based on N/S and E/W references
    if gps_data["lat_ref"] != "N":
        lat = -lat
    if gps_data["lon_ref"] != "E":
        lon = -lon

    formatted = f"{lat}° {gps_data['lat_ref']}, {lon}° {gps_data['lon_ref']}"
    return formatted, lat, lon

# Create Google Maps Link
Generates a Google Maps URL based on `latitude` and `longitude`.

In [7]:
def create_google_maps_link(lat, lon):
    """
    Creates a Google Maps link for the given coordinates.

    Args:
        lat (float): Latitude.
        lon (float): Longitude.

    Returns:
        str: A Google Maps link.
    """
    return f"https://www.google.com/maps?q={lat},{lon}"

# Example Usage
Demonstrates the usage of the functions to extract GPS coordinates from an image, format the coordinates, and create a Google Maps link.

In [8]:
image_path = "images/4.jpg"
exif_data = extract_exif(image_path)
gps_data = get_gps_info(exif_data)

if gps_data:
    formatted_coords, latitude, longitude = format_gps_coordinates(gps_data)
    print(f"GPS Coordinates: {formatted_coords}")
    print(f"Google Maps link: {create_google_maps_link(latitude, longitude)}")
else:
    print("No GPS information found.")


GPS Coordinates: 48.88197250555556° N, 2.3547717416666667° E
Google Maps link: https://www.google.com/maps?q=48.88197250555556,2.3547717416666667
